Set three variables for directories in the next cell.  Note that there shouldn't be a trailing slash after any of them, and these should be full paths to the original images, the directory to save the pickled numpy arrays, and the directory to save the images created from the pickle files, respectively.

In [ ]:

# original images directory
source_img_dir = "/Users/mcloney/Downloads/image-recognition-eval-master/images/original/item"
# (existing, preferably empty) directory for pickle file output
pickle_dir = "/Users/mcloney/Desktop/pickles"
# Directory to place the unpickled images
out_dir = "/Users/mcloney/Desktop/reconstituted"

In [1]:
import os
from scipy import ndimage
from PIL import Image
from six.moves import cPickle as pickle
import numpy as np
import scipy.misc
# image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def vectorize_image(folder, image, pixel_depth=None):
    
    image_file = os.path.join(folder, image)
    try:
        if not pixel_depth:
            pixel_depth = 255.0

        # normalize all values to between -0.5 and 0.5
        image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth

        return image_data
    except IOError as e:
        print('Could not read:', image_file, ':', e, '- skipping.')

In [2]:
def unnormalize_image(X, pixel_depth):
    x = (X * pixel_depth) + pixel_depth / 2
    return x.astype(int)

In [3]:
def reconstitute_image(in_pickle, out_image):
    with open(in_pickle, 'rb') as f:
        pkl = pickle.load(f)

    u_array = unnormalize_image(pkl, 255)
    scipy.misc.toimage(u_array).save(out_image)

In [4]:
def pickle_all_images(img_dir, img_file_list, pickle_dir):
    for img in img_file_list:
        print("Processing {}".format(img))
        pickled = vectorize_image(img_dir, img)
        outfile = ".".join([img.replace(".", "_"), "pickle"])
        with open(os.sep.join([pickle_dir, outfile]), 'wb') as f:
            pickle.dump(pickled, f)

In [5]:
def unpickle_all_images(pickle_dir, img_out_dir):
    for pfile in os.listdir(pickle_dir):
        if pfile[-7:].lower() == ".pickle":
            print("Processing {}".format(pfile))
            # nasty I know but it works :)
            file_ext = pfile[::-1][7:].split("_")[0][::-1]
            pkl_full_path = os.sep.join([pickle_dir, pfile])
            outfile = os.sep.join([img_out_dir, ".".join([pfile.replace(".", "_"), file_ext])])
            # print("reconstituting {} to {}".format(pkl_full_path, outfile))
            reconstitute_image(pkl_full_path, outfile)
        else:
            print("{} is not a pickle file, ignoring".format(pfile))

In [6]:
# MAIN CODE

all_objects = os.listdir(source_img_dir)
jpgs = [x for x in all_objects if x[-4:] == ".jpg" or x[-5:] == ".jpeg"]
pngs = [x for x in all_objects if x[-4:] == ".png"]
jsns = [x for x in all_objects if x[-5:] == ".json"]

# just because we know this is what we're looking for here:
assert len(jpgs) + len(pngs) == len(jsns) and len(jsns) > 0

print("*" * 100)
print("BEGIN PICKLING")
print("*" * 100)
print("\n" * 2)
pickle_all_images(source_img_dir, jpgs+pngs, pickle_dir)
print("\n" * 5)
print("*" * 100)
print("BEGIN RECONSTITUTING (UNPICKLING)")
print("*" * 100)
print("\n" * 5)
unpickle_all_images(pickle_dir, out_dir)


****************************************************************************************************
BEGIN PICKLING
****************************************************************************************************



Processing black-rim-glasses.jpg
Processing cello.jpg
Processing desktv.jpeg
Processing dogflowers.jpeg
Processing excavator.jpeg
Processing fastfood.jpg
Processing fish-real-1.jpg
Processing fish-real-2.jpg
Processing fish-real-held.jpg
Processing fish-real-held2.jpg
Processing ladybug.jpg
Processing stack-of-books.jpg
Processing tree-birch.jpg
Processing tree-palm.jpg
Processing tree-sugar-maple.jpg
Processing tree-yoshino-cherry.jpg
Processing violin.jpg
Processing VR-headset.jpg
Processing car.png
Processing fish-art-1.png
Processing guitar.png






****************************************************************************************************
BEGIN RECONSTITUTING (UNPICKLING)
************************************************************************************